In [442]:
import tensorflow as tf
import os
import keras

In [443]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv("train.csv")
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Id',axis = 1, inplace = True)
train.fillna(0,inplace=True)

test = pd.read_csv("test.csv")
test = test.select_dtypes(exclude=['object'])
ID = test.Id
test.fillna(0,inplace=True)
test.drop('Id',axis = 1, inplace = True)

print("")
print("List of features contained our dataset:",list(train.columns))

Shape of the train data with all features: (1460, 81)

Shape of the train data with numerical features: (1460, 38)

List of features contained our dataset: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [444]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 146
Number of rows without outliers: 1314


In [445]:
train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [446]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1314,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.235294,0.207668,0.062802,0.625,0.428571,0.953488,0.883333,0.12250,0.416274,0.0,...,0.000000,0.167582,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.403057
1,0.000000,0.255591,0.072904,0.500,0.857143,0.744186,0.433333,0.00000,0.576651,0.0,...,0.404891,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.340220
2,0.235294,0.217252,0.087396,0.625,0.428571,0.937984,0.866667,0.10125,0.286557,0.0,...,0.000000,0.115385,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.437966
3,0.294118,0.191693,0.072464,0.625,0.428571,0.271318,0.333333,0.00000,0.127358,0.0,...,0.000000,0.096154,0.824242,0.0,0.0,0.0,0.0,0.090909,0.00,0.243639
4,0.235294,0.268371,0.113835,0.750,0.428571,0.930233,0.833333,0.21875,0.386203,0.0,...,0.260870,0.230769,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.499638


In [447]:
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "SalePrice"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.SalePrice

# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training for submission
training_sub = training_set[col_train]

In [448]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
1232,0.235294,0.239617,0.074221,0.625,0.571429,0.914729,0.800000,0.00000,0.574882,0.0,...,0.343750,0.467033,0.0,0.0,0.0,0.0,0.000000,0.454545,0.00,0.556191
76,0.352941,0.000000,0.063505,0.625,0.428571,0.891473,0.766667,0.01375,0.000000,0.0,...,0.163043,0.197802,0.0,0.0,0.0,0.0,0.045161,0.363636,0.75,0.309966
962,0.235294,0.249201,0.091349,0.500,0.571429,0.682171,0.300000,0.00000,0.175708,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.363636,0.75,0.276221
433,0.294118,0.159744,0.010540,0.625,0.857143,0.271318,0.916667,0.00000,0.176297,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.454545,0.75,0.278548
1110,0.294118,0.306709,0.103927,0.375,0.428571,0.263566,0.000000,0.00000,0.000000,0.0,...,0.275815,0.109890,0.0,0.0,0.0,0.0,0.000000,0.545455,0.00,0.241049


In [449]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.regularizers import l1



In [450]:
seed = 7
np.random.seed(seed)
dropout=0.2
kernel_regularizer = l1(0.0001)
# Model
model = Sequential()
#model.add(Dense(200, input_dim=36, kernel_initializer='normal', activation='relu'))
model.add(Dense(200, input_dim=36, kernel_regularizer=kernel_regularizer, activation='relu'))
#model.add(Dropout(dropout))
#model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_regularizer=kernel_regularizer, activation='relu'))
#model.add(Dropout(dropout))
#model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_regularizer=kernel_regularizer, activation='relu'))
#model.add(Dropout(dropout))
#model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_regularizer=kernel_regularizer, activation='relu'))
#model.add(Dropout(dropout))
#model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(1, kernel_regularizer=kernel_regularizer))
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_152 (Dense)            (None, 200)               7400      
_________________________________________________________________
dense_153 (Dense)            (None, 100)               20100     
_________________________________________________________________
dense_154 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_155 (Dense)            (None, 25)                1275      
_________________________________________________________________
dense_156 (Dense)            (None, 1)                 26        
Total params: 33,851
Trainable params: 33,851
Non-trainable params: 0
_________________________________________________________________


In [451]:
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())
#model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False))
feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=300, batch_size=10)

Epoch 1/300
880/880 [==============================] - 1s 1ms/step - loss: 0.2669
Epoch 2/300
880/880 [==============================] - 0s 334us/step - loss: 0.2382
Epoch 3/300
880/880 [==============================] - 0s 330us/step - loss: 0.2142
Epoch 4/300
880/880 [==============================] - 0s 331us/step - loss: 0.1916
Epoch 5/300
880/880 [==============================] - 0s 328us/step - loss: 0.1701
Epoch 6/300
880/880 [==============================] - 0s 330us/step - loss: 0.1494
Epoch 7/300
880/880 [==============================] - 0s 339us/step - loss: 0.1309
Epoch 8/300
880/880 [==============================] - 0s 330us/step - loss: 0.1140
Epoch 9/300
880/880 [==============================] - 0s 334us/step - loss: 0.0971
Epoch 10/300
880/880 [==============================] - 0s 344us/step - loss: 0.0830
Epoch 11/300
880/880 [==============================] - 0s 337us/step - loss: 0.0704
Epoch 12/300
880/880 [==============================] - 0s 356us/step - loss

880/880 [==============================] - 0s 338us/step - loss: 0.0046
Epoch 97/300
880/880 [==============================] - 0s 338us/step - loss: 0.0047
Epoch 98/300
880/880 [==============================] - 0s 335us/step - loss: 0.0048
Epoch 99/300
880/880 [==============================] - 0s 336us/step - loss: 0.0048
Epoch 100/300
880/880 [==============================] - 0s 331us/step - loss: 0.0046
Epoch 101/300
880/880 [==============================] - 0s 331us/step - loss: 0.0045
Epoch 102/300
880/880 [==============================] - 0s 332us/step - loss: 0.0047
Epoch 103/300
880/880 [==============================] - 0s 342us/step - loss: 0.0046
Epoch 104/300
880/880 [==============================] - 0s 329us/step - loss: 0.0045
Epoch 105/300
880/880 [==============================] - 0s 333us/step - loss: 0.0045
Epoch 106/300
880/880 [==============================] - 0s 332us/step - loss: 0.0046
Epoch 107/300
880/880 [==============================] - 0s 335us/step 

880/880 [==============================] - 0s 333us/step - loss: 0.0037
Epoch 192/300
880/880 [==============================] - 0s 329us/step - loss: 0.0038
Epoch 193/300
880/880 [==============================] - 0s 331us/step - loss: 0.0038
Epoch 194/300
880/880 [==============================] - 0s 329us/step - loss: 0.0038
Epoch 195/300
880/880 [==============================] - 0s 344us/step - loss: 0.0037
Epoch 196/300
880/880 [==============================] - 0s 343us/step - loss: 0.0037
Epoch 197/300
880/880 [==============================] - 0s 395us/step - loss: 0.0037
Epoch 198/300
880/880 [==============================] - 0s 340us/step - loss: 0.0038
Epoch 199/300
880/880 [==============================] - 0s 330us/step - loss: 0.0037
Epoch 200/300
880/880 [==============================] - 0s 366us/step - loss: 0.0038
Epoch 201/300
880/880 [==============================] - 0s 397us/step - loss: 0.0038
Epoch 202/300
880/880 [==============================] - 0s 396us/st

880/880 [==============================] - 0s 342us/step - loss: 0.0037
Epoch 287/300
880/880 [==============================] - 0s 328us/step - loss: 0.0037
Epoch 288/300
880/880 [==============================] - 0s 351us/step - loss: 0.0036
Epoch 289/300
880/880 [==============================] - 0s 348us/step - loss: 0.0037
Epoch 290/300
880/880 [==============================] - 0s 339us/step - loss: 0.0037
Epoch 291/300
880/880 [==============================] - 0s 356us/step - loss: 0.0036
Epoch 292/300
880/880 [==============================] - 0s 350us/step - loss: 0.0036
Epoch 293/300
880/880 [==============================] - 0s 342us/step - loss: 0.0036
Epoch 294/300
880/880 [==============================] - 0s 347us/step - loss: 0.0036
Epoch 295/300
880/880 [==============================] - 0s 347us/step - loss: 0.0037
Epoch 296/300
880/880 [==============================] - 0s 341us/step - loss: 0.0037
Epoch 297/300
880/880 [==============================] - 0s 332us/st

In [452]:
# Evaluation on the test set created by train_test_split
model.evaluate(np.array(feature_cols), np.array(labels))

880/880 [==============================] - 1s 690us/step


0.0036421306007965045

In [453]:
# Predictions
feature_cols_test = testing_set[FEATURES]
labels_test = testing_set[LABEL].values

y = model.predict(np.array(feature_cols_test))
predictions = list(itertools.islice(y, testing_set.shape[0]))

In [454]:
y_predict = model.predict(np.array(test))

def to_submit(pred_y,name_out):
    y_predict = list(itertools.islice(pred_y, test.shape[0]))
    y_predict = pd.DataFrame(prepro_y.inverse_transform(np.array(y_predict).reshape(len(y_predict),1)), columns = ['SalePrice'])
    y_predict = y_predict.join(ID)
    y_predict.to_csv(name_out + '.csv',index=False)
    
to_submit(y_predict, "submit_prediction_7")

In [455]:
#Ldip